- **要加入急診的資料, 臨床資料要用來跑 baseline model**

## **Library**

In [1]:
import pandas as pd
import os 
import sys
import numpy as np
from tqdm import tqdm
from glob import glob
from datetime import timedelta

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.insert(0, src_path)  # 加入搜尋路徑最前面

from data_preprocess import(
    combine_datetime_MK,
    check_column,
    calculate_mews,
    calculate_SIRS,
    ED_list_process
)

## **Settings**

In [2]:
# %%[設定project路徑]
root_path = r"D:\python_code\CPD\GS"
os.chdir(root_path)
current_directory = os.getcwd()

print("目前的工作目錄是：", current_directory)
 
hospital= 'CMUH'

ED_data_path = r'D:\張裕鑫\RESEARCH\Databank\CMUH\ER 名單\processed_ED_list_csv'

variable_path= f'{root_path}\\data'
os.makedirs(variable_path, exist_ok=True)


目前的工作目錄是： D:\python_code\CPD\GS


## **Import files**

#### **CBC/CPD資料**

In [3]:
dtype= {'ID':int, 'Sample_ID':int, 'Age': int, }

df_CMUH_2021_origin=  pd.read_csv(rf'{variable_path}\CMUH_GS_2021.csv',
                                         dtype=dtype, parse_dates=['Report_Time'], encoding= 'utf-8-sig')


df_CMUH_2023_origin=  pd.read_csv(rf'{variable_path}\CMUH_GS_2023.csv',
                                         dtype=dtype, parse_dates=['Report_Time'], encoding= 'utf-8-sig')

#### **ED list**

In [4]:
raw_paths = glob(ED_data_path+r'\110*.csv') +glob(ED_data_path+r'\111*.csv')+glob(ED_data_path+r'\112*.csv') 

df_EDlist_origin= pd.concat([ED_list_process(p) for p in tqdm(raw_paths)])
df_EDlist_origin.reset_index(drop=True, inplace=True)

  0%|          | 0/29 [00:00<?, ?it/s]d:\python_code\CPD\GS\src\data_preprocess.py:76: DtypeWarning: Columns (39,40,41,46,101,114,119,121,125,128,129,130,131) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path, encoding='utf-8-sig')
  3%|▎         | 1/29 [00:00<00:07,  3.77it/s]d:\python_code\CPD\GS\src\data_preprocess.py:76: DtypeWarning: Columns (41,46,51,101,119,121,125,128,129,130,131) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path, encoding='utf-8-sig')
  7%|▋         | 2/29 [00:00<00:06,  4.19it/s]d:\python_code\CPD\GS\src\data_preprocess.py:76: DtypeWarning: Columns (39,40,92,114,119,121,123,125,128,129,130,131) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path, encoding='utf-8-sig')
 10%|█         | 3/29 [00:00<00:06,  4.24it/s]d:\python_code\CPD\GS\src\data_preprocess.py:76: DtypeWarning: Columns (39,83,101,116,119,121,125,1

- 先處理跑掉的ER list 格式

In [5]:
ER_total= df_EDlist_origin.copy()

In [6]:
# 找到包含 '急診' 或 '兒科' 的行
ER_total['ED_specialty_code'] = ER_total['ED_specialty_code'].astype(str) 

# 不要全部轉成str, 反正原本的數字再配對就會直接變成nan, 得到index後就把nan的行數刪掉, 留下來的理論上全部都要是True, 如果有False則代表有例外
# 例外就可以再去查是為什麼
mask = ER_total['ED_specialty_code'].str.contains(r'急診|兒科')
selected= ER_total[mask]
print (np.shape (selected)[0])

# column的範圍從 'GCS_E' 到倒数第二列
start_col = 'GCS_E'
end_col = ER_total.columns[-2]

start_idx = ER_total.columns.get_loc('GCS_E')
end_idx = ER_total.columns.get_loc(end_col)
cols_to_shift = ER_total.columns[start_idx:end_idx+1]

# 對符合條件的行進行處理
ER_total.loc[mask, cols_to_shift] = ER_total.loc[mask, cols_to_shift].shift(1, axis=1)

# 獲取符合条件的最後一個字符
ER_total.loc[mask, 'GCS_E'] = ER_total.loc[mask, 'Triage_depend'].str[-1]

9311


C:\Users\張裕鑫\AppData\Local\Temp\ipykernel_28200\973942761.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ER_total.loc[mask, cols_to_shift] = ER_total.loc[mask, cols_to_shift].shift(1, axis=1)
C:\Users\張裕鑫\AppData\Local\Temp\ipykernel_28200\973942761.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ER_total.loc[mask, cols_to_shift] = ER_total.loc[mask, cols_to_shift].shift(1, axis=1)
C:\Users\張裕鑫\AppData\Local\Temp\ipykernel_28200\973942761.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '

- **有一筆是地址往後移了一欄**

In [7]:

ER_total['首診主治醫師代碼'] = ER_total['首診主治醫師代碼'].astype(str) 
# 第一次不要全部轉成str, 反正原本的數字再配對就會直接變成nan, 得到index後就把nan的行數刪掉, 留下來的理論上全部都要是True, 如果有False則代表有例外
# 例外就可以再去查是為什麼

#確認是哪一種- 'nan' or np.nan, 這邊是'nan'
#ER_total['首診主治醫師代碼'].isin(['nan']).any().any()
#ER_total['首診主治醫師代碼'].isna().any().any()

ER_total['首診主治醫師代碼'] = ER_total['首診主治醫師代碼'].replace('nan', np.nan)
indices_to_shift = ER_total[ER_total['首診主治醫師代碼'].apply(check_column)].index
selected= ER_total.iloc[indices_to_shift,:]
print(len(indices_to_shift))

start_col = '首診主治醫師代碼'
start_idx = ER_total.columns.get_loc(start_col)
end_idx = np.shape(ER_total)[1]
cols_to_shift = ER_total.columns[start_idx:]


# 对符合条件的行进行处理，将列往前移动一个位置
X= ER_total.iloc[indices_to_shift, start_idx-1]+ ER_total.iloc[indices_to_shift, start_idx]  # 地址先合併好, 挑出來, 不然會被下面順移時蓋掉
ER_total.loc[indices_to_shift, cols_to_shift] = ER_total.loc[indices_to_shift, cols_to_shift].shift(-1, axis=1)
ER_total.iloc[indices_to_shift, start_idx-1] = X  #補回去

# 将最后一列设置为 NaN
ER_total.iloc[:, -1] = np.nan

#確認有沒有正確改好, print出來應該為0
indices_to_shift = ER_total[ER_total['首診主治醫師代碼'].apply(check_column)].index
selected= ER_total.loc[indices_to_shift,:]
print(len(indices_to_shift))

3


C:\Users\張裕鑫\AppData\Local\Temp\ipykernel_28200\3092612433.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ER_total.loc[indices_to_shift, cols_to_shift] = ER_total.loc[indices_to_shift, cols_to_shift].shift(-1, axis=1)
C:\Users\張裕鑫\AppData\Local\Temp\ipykernel_28200\3092612433.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ER_total.loc[indices_to_shift, cols_to_shift] = ER_total.loc[indices_to_shift, cols_to_shift].shift(-1, axis=1)
C:\Users\張裕鑫\AppData\Local\Temp\ipykernel_28200\3092612433.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future versi

0


In [8]:
# %%[change format]
ER_total['ED_Time'] = ER_total.apply(lambda row: combine_datetime_MK(row['EDdate'], row['EDtime']), axis=1)
EDtime_index = ER_total.columns.get_loc('EDtime')
ER_total.insert(EDtime_index + 1, 'ED_Time', ER_total.pop('ED_Time'))

ER_total['ED_DIS_date']= ER_total['ED_DIS_date'].astype(int)
ER_total['ED_DIS_time']= ER_total['ED_DIS_time'].astype(int)

ER_total['ED_DIS_Time'] = ER_total.apply(lambda row: combine_datetime_MK(row['ED_DIS_date'], row['ED_DIS_time']), axis=1)
ED_DIS_time_index = ER_total.columns.get_loc('ED_DIS_time')
ER_total.insert(ED_DIS_time_index + 1, 'ED_DIS_Time', ER_total.pop('ED_DIS_Time'))

## **Merge**

#### **processing**

In [9]:
df_CMUH_2021 = df_CMUH_2021_origin.copy()
df_CMUH_2023 = df_CMUH_2023_origin.copy()

df_CMUH_whole = pd.concat([df_CMUH_2021, df_CMUH_2023]).reset_index (drop=True)

In [10]:
df_CMUH_whole_2 = pd.merge(
    df_CMUH_whole,
    ER_total[
        [
            "ID",
            "ED_ID",
            "ED_Time",
            "ED_DIS_Time",
            "SBP",
            "DBP",
            "HR",
            "BT",
            "RR",
            "GCS_M",
            "Past_history",
            "ICD",
            "ICD_1",
            "ICD_2",
            "ICD_3",
            "ICD_4",
            "ICD_5"
        ]
    ],
    on="ID",
    how="left",
)

- CBC的Report_Time 要介於入院和出院間

In [11]:
df_CMUH_whole_2['ED_match'] = (
    (df_CMUH_whole_2['Report_Time'] >= df_CMUH_whole_2['ED_Time']) &
    (df_CMUH_whole_2['Report_Time'] <= df_CMUH_whole_2['ED_DIS_Time'] + timedelta(hours=4))
).astype(int)

df_CMUH_whole_2["delta_CBC_ED_Time"] = df_CMUH_whole_2['Report_Time'] - df_CMUH_whole_2['ED_Time']

- 把單一個Sample_ID 只有ED_match==0的抓出來

In [12]:
# Step 1: 找出只出現 ED_match == 0 的 Sample_ID
sample_ids_group = df_CMUH_whole_2.groupby('Sample_ID')['ED_match'].sum()
only_0_sample_ids = sample_ids_group[sample_ids_group == 0].index

# Step 2: 把這些 Sample_ID 的 ED_ID 設為 NaN
df_CMUH_whole_2.loc[df_CMUH_whole_2['Sample_ID'].isin(only_0_sample_ids), 'ED_ID'] = pd.NA


- 不在入院和出院的資料刪掉

In [13]:
# 篩選 ED_match == 1 的資料
df_matched = df_CMUH_whole_2[~(df_CMUH_whole_2['ED_ID'].isna())]

# 篩選 ED_match == 0 的資料
df_notmatched = df_CMUH_whole_2[(df_CMUH_whole_2['ED_ID'].isna())]

- 刪掉重複

In [14]:
df_matched_2 = df_matched [df_matched['ED_match']==1].reset_index(drop=True)

df_matched_2 = df_matched_2.sort_values(by=['Sample_ID',  'delta_CBC_ED_Time'], ascending=True)


#duplicate_rows = df_CMUH_whole_3.loc[df_CMUH_whole_3.duplicated(subset=['Sample_ID', 'Report_Time'], keep=False), ['Sample_ID', 'Report_Time', 'ED_Time', 'delta_CBC_ED_Time']]
df_matched_3 = df_matched_2.drop_duplicates(subset='Sample_ID', keep='first').reset_index(drop=True)


In [15]:
df_notmatched_2 = df_notmatched.drop_duplicates(subset='Sample_ID', keep='first').reset_index(drop=True)


- 合併回去

In [16]:
df_CMUH_whole_3= pd.concat([df_notmatched_2, df_matched_3], axis=0)

#### **Formatting**

In [17]:
VS_cols = ['SBP', 'DBP', 'HR', 'RR', 'BT', 'GCS_M']
df_CMUH_whole_3.loc[df_CMUH_whole_3['ED_ID'].isna(), VS_cols] = pd.NA

for col in VS_cols:
    invalid_vals = df_CMUH_whole_3[~df_CMUH_whole_3[col].apply(pd.to_numeric, errors='coerce').notna()][col].unique()
    print(f"{col} has invalid values: {invalid_vals}")


df_CMUH_whole_3[VS_cols] = df_CMUH_whole_3[VS_cols].apply(pd.to_numeric, errors='coerce')

SBP has invalid values: [<NA> nan '.' '...' '..']
DBP has invalid values: [<NA> nan '.' '...' '..']
HR has invalid values: [<NA> nan '.' '...' '..']
RR has invalid values: [<NA> nan '.' '0/E']
BT has invalid values: [<NA> nan '.']
GCS_M has invalid values: [<NA>]


- 切割出原本的train and validate set

In [18]:
# 設定切割時間點
cutoff_date = pd.to_datetime('2022-12-31')

# 分成兩個 DataFrame
df_2021_final = df_CMUH_whole_3[df_CMUH_whole_3['Report_Time'] <= cutoff_date].reset_index(drop=True)
df_2023_final = df_CMUH_whole_3[df_CMUH_whole_3['Report_Time'] > cutoff_date].reset_index(drop=True)


#### **算出MEWS**

In [19]:
df_2021_final.loc[:,'MEWS'] = df_2021_final.apply(calculate_mews, axis=1)
df_2023_final.loc[:,'MEWS'] = df_2023_final.apply(calculate_mews, axis=1)

#### **算出SIRS**

In [20]:
df_2021_final.loc[:,'SIRS'] = df_2021_final.apply(calculate_SIRS, axis=1)
df_2023_final.loc[:,'SIRS'] = df_2023_final.apply(calculate_SIRS, axis=1)

## **saving**

In [21]:
variable_path

'D:\\python_code\\CPD\\GS\\data'

In [22]:
df_2021_final.to_csv(rf"{variable_path}\CMUH_GS_2021_triage.csv", encoding= "utf-8-sig", index = False)
df_2023_final.to_csv(rf"{variable_path}\CMUH_GS_2023_triage.csv", encoding= "utf-8-sig", index = False)